<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inheritance: For Good or For Worse

## Subclassing Built-In Types Is Tricky

In [1]:
# Example 12-2. The __getitem__ of AnswerDict is bypassed by dict.update

class AnswerDict(dict):
  def __getitem__(self, key):
    return 42
  
ad = AnswerDict(a='foo')
ad['a']

42

In [5]:
dd = {}
dd.update(ad)
dd['a']

'foo'

In [6]:
dd

{'a': 'foo'}

In [0]:
# The dict.update method ignored our AnswerDict.__getitem__.

# The dict.update method ignored our AnswerDict.__getitem__.

### Remark

Subclassing built-in types like dict or list or str directly is errorprone because the built-in methods mostly ignore user-defined
overrides. Instead of subclassing the built-ins, derive your classes
from the collections module using UserDict, UserList, and
UserString, which are designed to be easily extended.

In [11]:
import collections

class DoppelDict2(collections.UserDict):
  def __setitem__(self, key, value):
    super().__setitem__(key, [value] * 2)
    
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [12]:
class AnswerDict2(collections.UserDict):
  def __getitem__(self, key):
    return 42
  
ad = AnswerDict2(a='foo')
ad['a']


42

In [13]:
d = {}
d.update(ad)
d['a']

42

In [14]:
d

{'a': 42}

### Remark

__MyWords__
When we try to inherit from built-in classes in Python, we cannot overrid the special methods which are implemented in C. For this mean, we should override collections.UserDick or Collections.UserList methods. 


The problem described in this section applies only to method delegation
within the C language implementation of the built-in types, and only affects userdefined classes derived directly from those types. If you subclass from a class coded in Python, such as UserDict or MutableMapping, you will not be troubled by this.

## Multiple Inheritance and Method Resolution Order

In [0]:
# Example 12-4. diamond.py: classes A, B, C, and D form
class A:
  def ping(self):
    print('ping:', self)
    
class B(A):
    def pong(self):
      print('pong:', self)
      
class C(A):
  def pong(self):
    print('PONG:', self)
    
class D(B, C):
  
  def ping(self):
    super().ping()
    print('post-ping:', self)
    
    
  def pingpong(self):
    self.ping()   
    super().ping() # The recommended way to delegate method calls to superclasses is the super() built-in function
    self.pong()
    super().pong()
    C.pong(self)

In [18]:
d = D()
d.pong()

pong: <__main__.D object at 0x7f801cd824a8>


In [21]:
C.pong(d) # You can always call a method on a superclass directly, passing the instance as an explicit argument.

PONG: <__main__.D object at 0x7f801cd824a8>


### Method Resolution Order (MRO)

The ambiguity of a call like d.pong() is resolved because Python follows a specific order
when traversing the inheritance graph. That order is called MRO: Method Resolution
Order. Classes have an attribute called \_\_mro\_\_ holding a tuple of references to the
superclasses in MRO order, from the current class all the way to the object class. For
the D class, this is the \_\_mro\_\_

In [22]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)